# Interop with SketchUp via `.dae` files

In this notebook, we will explore how to use the `SketchUpModel` facade to interact with exportable tqec SketchUp models. 

In [14]:
from tqec.sketchup.model import SketchUpModel
from tqec.sketchup.block import BlockType

## Library Blocks

All the library/template blocks are defined in the backend to be the building blocks for a model. A library block is defined by its geometries(faces) and materials(X/Z/H face type). For example, a `zxx` block is represented by its 6 faces as follows:
```py
Block(block_type=<BlockType.ZXX: 'zxx'>,
      faces=[Face(face_type=<FaceType.Z: 'Z'>,
                  width=1,
                  height=1,
                  normal_direction='X',
                  translation=(0, 0, 0)),
             Face(face_type=<FaceType.Z: 'Z'>,
                  width=1,
                  height=1,
                  normal_direction='X',
                  translation=(1, 0, 0)),
             Face(face_type=<FaceType.X: 'X'>,
                  width=1,
                  height=1,
                  normal_direction='Y',
                  translation=(0, 0, 0)),
             Face(face_type=<FaceType.X: 'X'>,
                  width=1,
                  height=1,
                  normal_direction='Y',
                  translation=(0, 1, 0)),
             Face(face_type=<FaceType.X: 'X'>,
                  width=1,
                  height=1,
                  normal_direction='Z',
                  translation=(0, 0, 0)),
             Face(face_type=<FaceType.X: 'X'>,
                  width=1,
                  height=1,
                  normal_direction='Z',
                  translation=(0, 0, 1))])
```

The definition of the library blocks specify the necessary "library" information when constructing the `.dae` file from backend. To be more specific, every `.dae` file constructed from the backend will have the same `<library_geometries>`, `<library_materials>` and `<library_nodes>` attributes.


## SketchUp Model

Now that we have defined the necessary library information for the model, we only need to add **instances** to the scene of the model. An instance block is defined to be a reference to a library block with some transformation(translation/scale) applied to it.

The shape of a instance is determined by its type and the `scale` property. And all the block is cube or cuboid. Thus we can infer the position of the instances by their connectivity. Therefore, it suffice to store the instances with a graph, which is encapsulated as the `SketchUpModel`. Note that we categorize all the block types into 2 categories: `cube` and `connector`. The cube instances are represented as the nodes in the graph, and the connector instances are represented as the edges in the graph. The following example shows how to construct a `SketchUpModel` that represents a CNOT gate.

<img src="https://github.com/QCHackers/tqec/assets/61700160/99b7429e-3f40-43ec-964a-79e08360c33d" width="400">

In [15]:
model = SketchUpModel()

# Construct a CNOT gate from scratch
zxz_cubes = [model.add_cube(BlockType.ZXZ) for _ in range(9)]
zxx_cube = model.add_cube(BlockType.ZXX)
for pair in [(0, 1), (1, 2), (2, 3), (5, 6), (6, 7), (7, 8)]:
    model.add_connector(BlockType.ZXO, zxz_cubes[pair[0]], zxz_cubes[pair[1]])
model.add_connector(BlockType.ZXO, zxx_cube, zxz_cubes[4])
model.add_connector(BlockType.ZOX, zxz_cubes[1], zxx_cube)
model.add_connector(BlockType.OXZ, zxz_cubes[4], zxz_cubes[7])

# Save the model as a .dae file
model.write("assets/cnot_from_scratch.dae")

# Import model
model = SketchUpModel.from_dae_file("assets/cnot_from_scratch.dae")
print(f"SketchUp model with {model.num_cubes} cubes and {model.num_connectors} connectors")

SketchUp model with 10 cubes and 9 connectors


## Rescale the connectors

By the canonical definition of library blocks, all the connectors have length 2 along the connecting direction. The length can be rescaled by a factor scale to make the connector length as `2*scale`. The rescaling will have effect on the whole model when infering the positions of the blocks.

In [16]:
model = SketchUpModel.from_dae_file("assets/cnot_from_scratch.dae")
model.set_scale_for_all_connectors(0.5)
model.write("assets/cnot_from_scratch_half_scale.dae")

model.set_scale_for_all_connectors(2)
model.write("assets/cnot_from_scratch_twice_scale.dae")

We manually import all the three scales of the same models into SketchUp and get:

<img src="https://github.com/QCHackers/tqec/assets/61700160/4b59b231-2b71-4a77-ad83-cc51f6d82f53" width="600">